In [ ]:
%pip install langchain
%pip install pinecone-client
%pip install pypdf
%pip install langchain-text-splitters

In [34]:
import os
from dotenv import load_dotenv

load_dotenv()
HUGGING_FACE_API = os.getenv('HUGGING_FACE_API')
PINECONE_API_KEY = os.getenv('PINECONE_API')

In [5]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("pdfs/")
data = loader.load()

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(

    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

text_chunks = text_splitter.split_documents(data)

In [67]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HUGGING_FACE_API, model_name="sentence-transformers/all-MiniLM-l6-v2"
)


In [68]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "test"

index = pc.Index(index_name, metric="cosine", host="https://test-f6d2zst.svc.gcp-starter.pinecone.io")

In [70]:
from langchain_pinecone import PineconeVectorStore

pvs = PineconeVectorStore(pinecone_api_key=PINECONE_API_KEY)

docsearch = pvs.from_documents([t for t in text_chunks], embedding=embeddings, index_name=index_name)

ValueError: Embedding must be provided